In [1]:
%%time
# Create SparkSession from builder
import pyspark
from pyspark.sql import SparkSession
from functools import reduce  # For Python 3.x
from pyspark.sql import DataFrame
from pyspark.sql.functions import isnan, when, count, col
from pyspark.sql import Row
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
import pyspark.pandas as ps
from pyspark.sql.functions import monotonically_increasing_id, row_number
from pyspark.sql.window import Window
import pyspark.pandas as ps
from pyspark.conf import SparkConf
from pyspark.sql import SparkSession


import psycopg2

# connection establishment
conn = psycopg2.connect(
database="postgres",
	user='postgres',
	password='vivek@3011',
	host='localhost',
	port= '5432'
)

conn.autocommit = True

# Creating a cursor object
cursor = conn.cursor()

# query to create a database
sql = ''' CREATE DATABASE yellow_tripdata_analysis; ''';

# executing above query
cursor.execute(sql)
#print("Database has been created successfully !!");

# Closing the connection
conn.commit()

conf = pyspark.SparkConf().setAll([('spark.driver.extraClassPath', 'E:/IITI/data_analytics/crio_project/big_data/data_bricks/New folder/postgresql-42.3.7.jar'), \
                                   ('spark.executor.extraClassPath', 'E:/IITI/data_analytics/crio_project/big_data/data_bricks/New folder/postgresql-42.3.7.jar'),\
                                   ("spark.jars", "E:/IITI/data_analytics/crio_project/big_data/data_bricks/New folder/postgresql-42.3.7.jar"),\
                                   ("spark.executor.extraLibraryPath","E:/IITI/data_analytics/crio_project/big_data/data_bricks/New folder/postgresql-42.3.7.jar"),\
                                    ])


def unionAll(*dfs):
    return reduce(DataFrame.unionAll, dfs)


from pyspark.sql.functions import monotonically_increasing_id
def get_mode(df):
    column_lst = df.columns
    res = [df.select(i).groupby(i).count().orderBy("count", ascending=False) for i in column_lst]
    df_mode = res[0].limit(1).select(column_lst[0]).withColumn("temp_name_monotonically_increasing_id", monotonically_increasing_id())
    
    for i in range(1, len(res)):
        df2 = res[i].limit(1).select(column_lst[i]).withColumn("temp_name_monotonically_increasing_id", monotonically_increasing_id())
        df_mode = df_mode.join(df2, (df_mode.temp_name_monotonically_increasing_id == df2.temp_name_monotonically_increasing_id)).drop(df2.temp_name_monotonically_increasing_id)
        
    return df_mode.drop("temp_name_monotonically_increasing_id")





spark = SparkSession.builder.config(conf=conf) \
                    .master("local") \
                    .appName('yellow_tripdata_analysis') \
                    .getOrCreate()
parquets=["data_parquet\yellow_tripdata_2019\yellow_tripdata_2019-01.parquet",
          "data_parquet\yellow_tripdata_2019\yellow_tripdata_2019-02.parquet",
          "data_parquet\yellow_tripdata_2019\yellow_tripdata_2019-03.parquet",
          "data_parquet\yellow_tripdata_2019\yellow_tripdata_2019-04.parquet",
          "data_parquet\yellow_tripdata_2019\yellow_tripdata_2019-05.parquet",
          "data_parquet\yellow_tripdata_2019\yellow_tripdata_2019-06.parquet",
          "data_parquet\yellow_tripdata_2019\yellow_tripdata_2019-07.parquet",
          "data_parquet\yellow_tripdata_2019\yellow_tripdata_2019-08.parquet",
          "data_parquet\yellow_tripdata_2019\yellow_tripdata_2019-09.parquet",
          "data_parquet\yellow_tripdata_2019\yellow_tripdata_2019-10.parquet",
          "data_parquet\yellow_tripdata_2019\yellow_tripdata_2019-11.parquet",
          "data_parquet\yellow_tripdata_2019\yellow_tripdata_2019-12.parquet"]

data=[]
for i in parquets:
    d = spark.read.option("header","true").parquet(i,inferSchema=True)
    data.append(d)
    
#Merge all the dataframes in list
df_complete=unionAll(*data)
#print("Total Rows and Columns:",(df_complete.count(), len(df_complete.columns)))
columns=df_complete.columns

#Mode values for the DataFrame
mode_values=get_mode(df_complete)


pd=ps.DataFrame(df_complete)
df_corr=pd.to_spark()
df_corr=df_corr.withColumn("date_format",to_date(df_complete.tpep_pickup_datetime,"MM-dd-yyyy"))
df_corr=df_corr.withColumn("year",year(df_corr.date_format))
a=df_corr.first()["year"]
b=str(a)
df_corr=df_corr.where(df_corr.date_format.contains(b))
# df_corr=df_corr.where(df_corr.date_format.contains("2019"))

mode=[]
for i in range(len(df_complete.columns)): 
    mode.append(mode_values.collect()[0][i])
check=zip(df_complete.columns,mode)
mydict=dict(check)
a_dict = {key: mydict[key] for key in mydict if (key != 'VendorID' and key !='tpep_pickup_datetime' and key !='tpep_dropoff_datetime')} 
if a_dict["airport_fee"]==None:
          a_dict["airport_fee"]="Unknown"
df_corr=df_corr.fillna(a_dict)



#I made a little helper function for this that might help some people out.

import re

from functools import partial

def rename_cols(agg_df, ignore_first_n=1):
    """changes the default spark aggregate names `avg(colname)` 
    to something a bit more useful. Pass an aggregated dataframe
    and the number of aggregation columns to ignore.
    """
    delimiters = "(", ")"
    split_pattern = '|'.join(map(re.escape, delimiters))
    splitter = partial(re.split, split_pattern)
    split_agg = lambda x: '_'.join(splitter(x))[0:-ignore_first_n]
    renamed = map(split_agg, agg_df.columns[ignore_first_n:])
    renamed = zip(agg_df.columns[ignore_first_n:], renamed)
    for old, new in renamed:
        agg_df = agg_df.withColumnRenamed(old, new)
    return agg_df



week_level=df_corr.drop('RatecodeID',
 'store_and_fwd_flag',
 'PULocationID',
 'DOLocationID',)
week_level=week_level.withColumn("week_date",date_trunc("week",week_level.date_format))
column_change_week_level=week_level.groupBy("VendorID","week_date").sum()
final1=rename_cols(column_change_week_level)
final_week_level=final1.drop("sum_VendorID","sum_payment_type","sum_passenger_count")



month_level=df_corr.drop('RatecodeID',
 'store_and_fwd_flag',
 'PULocationID',
 'DOLocationID',)
month_level=month_level.withColumn("Month",month(month_level.date_format))
column_change_month_level=month_level.groupBy("VendorID","Month").sum()
final2=rename_cols(column_change_month_level)
final_month_level=final2.drop("sum_VendorID","sum_payment_type","sum_Month","sum_passenger_count")


avg_month=df_corr.drop('RatecodeID',
 'store_and_fwd_flag',
 'PULocationID',
 'DOLocationID',)
avg_month=avg_month.withColumn("Month",month(month_level.date_format))
final_avg_month=avg_month.groupby("VendorID","Month").agg({"congestion_surcharge":"avg"}).alias('Avg_congestion_surcharge').orderBy("Month", ascending=False)
  


pass_month=df_corr.drop('RatecodeID',
 'store_and_fwd_flag',
 'PULocationID',
 'DOLocationID',)
pass_month=pass_month.withColumn("Month",month(month_level.date_format))
final_pass_month=pass_month.groupby("VendorID","Month").agg({"passenger_count":"sum"}).alias('Total_passenger_count').orderBy("Month")   


trip_hour=df_corr.drop('RatecodeID',
 'store_and_fwd_flag',
 'PULocationID',
 'DOLocationID',)
trip_hour=trip_hour.withColumn("hour", hour(trip_hour.tpep_pickup_datetime))
trip1=trip_hour.groupby("hour").agg(count("tpep_pickup_datetime").alias('total_trip_count')).orderBy("hour")
import pyspark.sql.functions as f
from pyspark.sql.window import Window
trip1 = trip1.withColumn('percent', f.col('total_trip_count')/f.sum('total_trip_count').over(Window.partitionBy()))
final_trip_hour=trip1.orderBy('percent', ascending=False)


payment_month=df_corr.drop('RatecodeID',
 'store_and_fwd_flag',
 'PULocationID',
 'DOLocationID',)
payment_month=payment_month.withColumn("Month",month(month_level.date_format))
from pyspark.sql.functions import when
payment_month = payment_month.withColumn("payment_type_name", 
              when(payment_month.payment_type == 0, "Credit card")
             .when(payment_month.payment_type == 1, "Cash")
             .when(payment_month.payment_type == 2, "No charge")                       
             .when(payment_month.payment_type == 3, "Dispute")  
             .when(payment_month.payment_type == 4, "Unknown")  
             .when(payment_month.payment_type == 5, "Voided trip")  
             .otherwise(payment_month.payment_type))
payment_month1=payment_month.groupby("Month","payment_type_name").agg(count("payment_type_name").alias('payment_type_name_count')).orderBy("Month")
import pyspark.sql.functions as f
from pyspark.sql.window import Window
payment_month1 = payment_month1.withColumn('percent', f.col('payment_type_name_count')/f.sum('payment_type_name_count').over(Window.partitionBy()))
final_payment_month=payment_month1.orderBy('percent', ascending=False)





import pandas as pd
#import findspark
#findspark.init()
import pyspark
from pyspark import SparkContext
from pyspark.sql import SQLContext 
#sc = SparkContext("local", "App Name")
#sql = SQLContext(sc)
from pyspark.sql.window import Window
from pyspark.sql.functions import rank, col

payment_month_top=payment_month.groupby("Month","payment_type_name").agg(count("payment_type_name").alias('payment_type_name_count')).orderBy("Month")
window = Window.partitionBy(payment_month_top['Month']).orderBy(payment_month_top['payment_type_name_count'].desc())
final_payment_month_top=payment_month_top.select('*', rank().over(window).alias('rank')).filter(col('rank') <= 3 ) 





final_week_level.select("*").write.format("jdbc")\
    .option("url", "jdbc:postgresql://localhost:5432/yellow_tripdata_analysis") \
    .option("driver", "org.postgresql.Driver").option("dbtable", "Weekly_Sales_Aggr") \
    .option("user", "postgres").option("password", "vivek@3011").save()

final_month_level.select("*").write.format("jdbc")\
    .option("url", "jdbc:postgresql://localhost:5432/yellow_tripdata_analysis") \
    .option("driver", "org.postgresql.Driver").option("dbtable", "Monthly_Sales_Aggr") \
    .option("user", "postgres").option("password", "vivek@3011").save()

final_avg_month.select("*").write.format("jdbc")\
    .option("url", "jdbc:postgresql://localhost:5432/yellow_tripdata_analysis") \
    .option("driver", "org.postgresql.Driver").option("dbtable", "Avg_monthly_surcharge") \
    .option("user", "postgres").option("password", "vivek@3011").save()

final_pass_month.select("*").write.format("jdbc")\
    .option("url", "jdbc:postgresql://localhost:5432/yellow_tripdata_analysis") \
    .option("driver", "org.postgresql.Driver").option("dbtable", "Total_Passengers_Monthly") \
    .option("user", "postgres").option("password", "vivek@3011").save()

final_trip_hour.select("*").write.format("jdbc")\
    .option("url", "jdbc:postgresql://localhost:5432/yellow_tripdata_analysis") \
    .option("driver", "org.postgresql.Driver").option("dbtable", "Trips_Hourly") \
    .option("user", "postgres").option("password", "vivek@3011").save()

final_payment_month.select("*").write.format("jdbc")\
    .option("url", "jdbc:postgresql://localhost:5432/yellow_tripdata_analysis") \
    .option("driver", "org.postgresql.Driver").option("dbtable", "Payment_Dist_Monthly") \
    .option("user", "postgres").option("password", "vivek@3011").save()

final_payment_month_top.select("*").write.format("jdbc")\
    .option("url", "jdbc:postgresql://localhost:5432/yellow_tripdata_analysis") \
    .option("driver", "org.postgresql.Driver").option("dbtable", "Top_Payment_Monthly") \
    .option("user", "postgres").option("password", "vivek@3011").save()






E:\Anaconda\envs\py3env\lib\site-packages\pyspark\pandas\utils.py:975: PandasAPIOnSparkAdviceWarning: If `index_col` is not specified for `to_spark`, the existing index is lost when converting to Spark DataFrame.
  warnings.warn(message, PandasAPIOnSparkAdviceWarning)


CPU times: total: 2.64 s
Wall time: 11min 49s


In [1]:
import psycopg2

# connection establishment
conn = psycopg2.connect(
database="postgres",
	user='postgres',
	password='vivek@3011',
	host='localhost',
	port= '5432'
)

conn.autocommit = True

# Creating a cursor object
cursor = conn.cursor()

# query to create a database
sql = ''' CREATE DATABASE final; ''';

# executing above query
cursor.execute(sql)
print("Database has been created successfully !!");

# Closing the connection
conn.close()


Database has been created successfully !!


In [6]:
df1 = spark.read \
    .format("jdbc") \
    .option("url", "jdbc:postgresql://localhost:5432/supermart-DB") \
    .option("dbtable", "customer") \
    .option("user", "postgres") \
    .option("password", "vivek@3011") \
    .option("driver", "org.postgresql.Driver") \
    .load()

In [7]:
df1.show()

+-----------+------------------+-----------+---+-------------+---------------+--------------+-----------+-------+
|customer_id|     customer_name|    segment|age|      country|           city|         state|postal_code| region|
+-----------+------------------+-----------+---+-------------+---------------+--------------+-----------+-------+
|   CG-12520|       Claire Gute|   Consumer| 67|United States|      Henderson|      Kentucky|      42420|  South|
|   DV-13045|   Darrin Van Huff|  Corporate| 31|United States|    Los Angeles|    California|      90036|   West|
|   SO-20335|    Sean O'Donnell|   Consumer| 65|United States|Fort Lauderdale|       Florida|      33311|  South|
|   BH-11710|   Brosina Hoffman|   Consumer| 20|United States|    Los Angeles|    California|      90032|   West|
|   AA-10480|      Andrew Allen|   Consumer| 50|United States|        Concord|North Carolina|      28027|  South|
|   IM-15070|      Irene Maddox|   Consumer| 66|United States|        Seattle|    Washin

In [9]:
df_corr.select("*").write.format("jdbc")\
    .option("url", "jdbc:postgresql://localhost:5432/yellow_tripdata_2019") \
    .option("driver", "org.postgresql.Driver").option("dbtable", "complete_data") \
    .option("user", "postgres").option("password", "vivek@3011").save()

In [ ]:
final_list=[final_week_level,final_month_level,final_avg_month,final_pass_month,final_trip,final_payment_month,fianl_payment_month_top]
for i in final_list:
    

In [4]:
from pyspark.sql import SparkSession

spark = SparkSession \
    .builder \
    .appName("Python Spark SQL basic example") \
    .config("spark.jars", "E:/IITI/data_analytics/cspark.jars.packagesrio_project/big_data/data_bricks/New folder/postgresql-42.3.7.jar") \
    .getOrCreate()

df = spark.read \
    .format("jdbc") \
    .option("url", "jdbc:postgresql://localhost:5432/supermart-DB") \
    .option("dbtable", "customer") \
    .option("user", "postgres") \
    .option("password", "vivek@3011") \
    .option("driver", "org.postgresql.Driver") \
    .load()

df.printSchema()

root
 |-- customer_id: string (nullable = true)
 |-- customer_name: string (nullable = true)
 |-- segment: string (nullable = true)
 |-- age: integer (nullable = true)
 |-- country: string (nullable = true)
 |-- city: string (nullable = true)
 |-- state: string (nullable = true)
 |-- postal_code: long (nullable = true)
 |-- region: string (nullable = true)



In [4]:
studentDf.select("id","name","marks").write.format("jdbc")\
    .option("url", "jdbc:postgresql://localhost:5432/yellow_tripdata_2019") \
    .option("driver", "org.postgresql.Driver").option("dbtable", "complete_data") \
    .option("user", "postgres").option("password", "vivek@3011").save()

Py4JJavaError: An error occurred while calling o60.save.
: java.lang.ClassNotFoundException: org.postgresql.Driver
	at java.base/java.net.URLClassLoader.findClass(URLClassLoader.java:445)
	at java.base/java.lang.ClassLoader.loadClass(ClassLoader.java:587)
	at java.base/java.lang.ClassLoader.loadClass(ClassLoader.java:520)
	at org.apache.spark.sql.execution.datasources.jdbc.DriverRegistry$.register(DriverRegistry.scala:46)
	at org.apache.spark.sql.execution.datasources.jdbc.JDBCOptions.$anonfun$driverClass$1(JDBCOptions.scala:101)
	at org.apache.spark.sql.execution.datasources.jdbc.JDBCOptions.$anonfun$driverClass$1$adapted(JDBCOptions.scala:101)
	at scala.Option.foreach(Option.scala:407)
	at org.apache.spark.sql.execution.datasources.jdbc.JDBCOptions.<init>(JDBCOptions.scala:101)
	at org.apache.spark.sql.execution.datasources.jdbc.JdbcOptionsInWrite.<init>(JDBCOptions.scala:229)
	at org.apache.spark.sql.execution.datasources.jdbc.JdbcOptionsInWrite.<init>(JDBCOptions.scala:23